In [ ]:
import os
import torch
import torchaudio
import pandas as pd
import numpy as np
from tqdm import tqdm
from scipy import signal
import matplotlib.pyplot as plt

# Change to root directory
os.chdir("..")
from src.utils import *

print("Current Working Directory:", os.getcwd())

# Define filter types and window sizes to use
filter_types = ["gaussian", "median"]
window_sizes = [12, 25, 50]  # in milliseconds

# Setup data paths
data_dir = "data"
data_type = "noisy"
noise_types = ["gaussian", "poisson", "rayleigh"]
snrs = [0, 5, 10, 15, 20]
output_subdir = "window_smoothing"


# Create a function to process and filter the dataset
def process_dataset_with_window_filter(
    noisy_dataset, data_dir, subdir, noise_type, snr, filter_type, window_size
):
    """
    Process a dataset applying a window filter and save the results

    Args:
        noisy_dataset: DataFrame containing paths to noisy audio files
        data_dir: Root data directory
        subdir: Output subdirectory
        noise_type: Type of noise in the dataset
        filter_type: Type of filter to apply ('gaussian' or 'median')
        window_size: Size of the filter window in milliseconds
    """
    dataset_name = f"vox1_test_wav_snr_{snr}_{noise_type}"
    output_dir = os.path.join(
        data_dir, subdir, f"filter_{filter_type}_{window_size}ms", noise_type, dataset_name
    )
    os.makedirs(output_dir, exist_ok=True)

    print(f"Processing with {filter_type} filter, window size {window_size}ms")

    for index, row in tqdm(
        noisy_dataset.iterrows(),
        total=noisy_dataset.shape[0],
        desc=f"Processing {noise_type} SNR {snr} with {filter_type}_{window_size}ms",
    ):
        noisy_path = row["path"]

        # Load audio
        audio, sample_rate = torchaudio.load(noisy_path)

        # Apply window filter
        filtered_audio = apply_window_filter(
            audio, filter_type, window_size, sample_rate
        )

        # Create output directory structure
        person_subdir = os.path.join(output_dir, row["person_id"])
        os.makedirs(person_subdir, exist_ok=True)

        # Save filtered audio
        filtered_audio_path = os.path.join(person_subdir, os.path.basename(noisy_path))
        torchaudio.save(filtered_audio_path, filtered_audio, sample_rate)


# Process all datasets with all filter combinations
for noise_type in noise_types:
    for snr in snrs:
        dataset_name = f"vox1_test_segments_snr_{snr}_noisy_{noise_type}"

        noisy_dt_path = os.path.join(data_dir, data_type, noise_type, dataset_name)
        if not os.path.exists(noisy_dt_path):
            print(f"Directory {noisy_dt_path} does not exist")
            continue

        print(f"Processing directory: {noisy_dt_path}")
        noisy_dataset = scan_directory_voxceleb2(noisy_dt_path)

        for filter_type in filter_types:
            for window_size in window_sizes:
                process_dataset_with_window_filter(
                    noisy_dataset,
                    data_dir,
                    output_subdir,
                    noise_type,
                    snr,    
                    filter_type,
                    window_size,
                )

# Process background noise datasets as well
noise_types = ["AirConditioner", "Babble", "Neighbor"]
data_type = "noisy_bg\\vox1_test_wav_bq_noise"

for noise_type in noise_types:
    for snr in snrs:
        dataset_name = f"vox1_test_wav_snr_{snr}_{noise_type}"

        noisy_dt_path = os.path.join(data_dir, data_type, noise_type, dataset_name)
        if not os.path.exists(noisy_dt_path):
            print(f"Directory {noisy_dt_path} does not exist")
            continue

        print(f"Processing directory: {noisy_dt_path}")
        noisy_dataset = scan_directory_voxceleb2(noisy_dt_path)

        for filter_type in filter_types:
            for window_size in window_sizes:
                process_dataset_with_window_filter(
                    noisy_dataset,
                    data_dir,
                    output_subdir,
                    noise_type,
                    snr,
                    filter_type,
                    window_size,
                )


print("Processing complete!")

Current Working Directory: c:\Users\filip\Documents\code\SV-eval
Processing directory: data\noisy\gaussian\vox1_test_segments_snr_0_noisy_gaussian
Processing with gaussian filter, window size 12ms


Processing gaussian SNR 0 with gaussian_12ms:   7%|▋         | 649/9119 [00:12<02:38, 53.33it/s]


KeyboardInterrupt: 